# Geokr

## Get config

In [1]:
import json

with open('.config') as fp:
    config = json.load(fp)

## Flickr class

In [2]:
import requests
import json

class Flickr:
    request_baseurl = 'https://api.flickr.com/services/rest/'
    jsonDecoder = json.JSONDecoder()
    
    def __init__(self, api_key):
        self.api_key = api_key

    def request(self, method, args):
        url = self.request_baseurl + '?api_key=' + self.api_key + '&format=json&method=' + method + '&' + '&'.join([k+'='+str(v) for k,v in args.iteritems()])
        return self.jsonDecoder.decode(requests.get(url).content[len("jsonFlickrApi("):-1])

    def get_photo_url(farm_id, server_id, photo_id):
        return "https://farm" + farm_id + ".staticflickr.com/" + server_id + "/" + photo_id + "_" + secret + ".jpg"


## Get recents photos

### Prepare requests

In [3]:
flickr = Flickr(config['API_KEY'])
data = []
nb_page = 10
method = 'flickr.photos.getRecent'
args = {'per_page':500,'extras':'geo'}

### Send and parse requests

In [4]:
for page in range(nb_page):
    args['page'] = page + 1
    data += flickr.request(method, args)["photos"]["photo"]

### Fetch results

In [5]:
print "%d photos found" % len(data)
geotagged = []

for photo in data:
    if photo["longitude"] != 0 or photo["latitude"] != 0:
        geotagged.append(photo)

print "%d geotagged photos found" % len(geotagged)

5000 photos found
509 geotagged photos found


## Get geotagged photos

### Prepare requests

In [6]:
flickr = Flickr(config['API_KEY'])
data = []
nb_page = 8
method = 'flickr.photos.search'
args = {
    'has_geo': 1,
    'per_page': 250,
    'extras': 'geo,date_taken',
    'min_taken_date': '2015-01-01',
    'max_taken_date': '2016-01-01'
}

### Send and parse requests

In [7]:
import json

for page in range(nb_page):
    args['page'] = page + 1
    data += flickr.request(method, args)["photos"]["photo"]

### Fetch results

In [8]:
print "%d photos found" % len(data)

1250 photos found
